In [201]:
from numpy import ceil, log2, floor, pi
import json
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, qpy
sv_backend = Aer.get_backend('statevector_simulator')

In [202]:
def test_loader_1(file_path):
    # make it faster ...
    with open(file_path, "r") as file:

        for row_id, row in enumerate(file):
            if row_id == 0:
                n, m = row.split()
                n, m = int(n), int(m)
            else:
                array = [int(x) for x in row.split()]

    return n, m, array

In [203]:
def test_loader_2(file_path):
    # make it faster ...
    n, rotations = None, []

    with open(file_path, "r") as file:
        for row_id, row in enumerate(file):
            if row_id == 0:
                n = int(row)
            else:
                r_type, value = row.split()
                value = float(value)
                rotations.append((r_type, value))

    return n, rotations

In [204]:
INPUT_PATH = "tests/inputs/"
OUTPUT_PATH = "tests/outputs/"

In [205]:
def get_qram(N, M, array):
    index = 0
    index_size = int(ceil(log2(N)))
    value_size = int(floor(log2(M))) + 1

    index_reg = QuantumRegister(name="index", size=index_size)
    value_reg = QuantumRegister(name="value", size=value_size)

    circ = QuantumCircuit(index_reg, value_reg, name="oracle")
    circ.h(index_reg)
    # to store circuits for previously seen elements
    circ_cache = {}

    def get_elem_circ(elem, idx):
        # analyse this element
        if elem in circ_cache:
            elem_circ = circ_cache[elem]
        else:
            bin_rep = bin(elem)[2:]
            elem_circ = QuantumCircuit(value_size, name=f"x_{idx}")
            bin_rep = bin_rep[::-1]

            for i, bit in enumerate(bin_rep):
                if bit == "1":
                    elem_circ.x(i)
            circ_cache[elem] = elem_circ

        elem_circ = elem_circ.control(
            num_ctrl_qubits=index_size, ctrl_state=idx)

        return elem_circ

    for elem in array:
        circuit = get_elem_circ(elem, index)
        circuit.name = f"x_{index}"
        circ.compose(circuit, inplace=True)
        index += 1

    del circ_cache

    return circ

In [206]:
def get_qram_rotations(N, rotations):
    index = 0
    index_size = int(ceil(log2(N)))
    value_size = 1

    index_reg = QuantumRegister(name="index", size=index_size)
    value_reg = QuantumRegister(name="value", size=value_size)

    circ = QuantumCircuit(index_reg, value_reg, name="oracle")
    circ.h(index_reg)
    # to store circuits for previously seen elements

    def get_elem_circ(idx, rot_type, angle):
        # analyse this element

        elem_circ = QuantumCircuit(value_size, name=f"{rot_type}_{round(angle,4)}")
        if rot_type == 'x':
            elem_circ.rx(2*pi * angle, 0)
        elif rot_type == 'y':
            elem_circ.ry(2*pi*angle, 0)
        else:
            elem_circ.rz(2*pi*angle, 0)

        elem_circ = elem_circ.control(
            num_ctrl_qubits=index_size, ctrl_state=idx)

        return elem_circ

    for elem in rotations:
        rot_type, angle = elem
        circuit = get_elem_circ(index, rot_type, angle)
        circuit.name = f"x_{index}"
        circ.compose(circuit, inplace=True)
        index += 1

    return circ

In [207]:
def output_saver(file_path, circ):
#     circ.qasm(filename = file_path)
    with open(file_path, 'wb') as file:
        qpy.dump(circ, file)

In [179]:
class answer_gen_1:
    ip_task_path = INPUT_PATH + "task-1-"
    op_task_path = OUTPUT_PATH + "task-1-"
    total_tests = 10

    @classmethod
    def generate_answer(cls):
        for test in range(answer_gen_1.total_tests):
            ip_test_path = cls.ip_task_path + str(test) + ".txt"
            n, m, array = test_loader_1(ip_test_path)

            qram = get_qram(n, m, array)
            print("Path : ", ip_test_path)
            print(array)
            print("Statevector is : ")

            sv = execute(qram, sv_backend).result().get_statevector()
            sv = sv.to_dict()
            print(sv)

            print()

            # store to the outputs
            op_test_path = cls.op_task_path + str(test) + ".qpy"
            output_saver(op_test_path, qram)

In [ ]:
answer_gen_1.generate_answer()

In [181]:
class answer_gen_2:
    ip_task_path = INPUT_PATH + "task-2-"
    op_task_path = OUTPUT_PATH + "task-2-"
    total_tests = 10

    @classmethod
    def generate_answer(cls):
        for test in range(answer_gen_2.total_tests):
            ip_test_path = cls.ip_task_path + str(test) + ".txt"
            n, m, array = test_loader_1(ip_test_path)

            qram = get_qram(n, m, array)
            print("Path : ", ip_test_path)
            print(array)
            print("Statevector is : ")

            sv = execute(qram, sv_backend).result().get_statevector()
            sv = sv.to_dict()
            print(sv)

#             print()

            # store to the outputs
            op_test_path = cls.op_task_path + str(test) + ".qpy"
            output_saver(op_test_path, qram)

In [ ]:
answer_gen_2.generate_answer()

In [184]:
class answer_gen_3:
    ip_task_path = INPUT_PATH + "task-3-"
    op_task_path = OUTPUT_PATH + "task-3-"
    total_tests = 10

    @classmethod
    def generate_answer(cls):
        for test in range(answer_gen_3.total_tests):
            ip_test_path = cls.ip_task_path + str(test) + ".txt"
            n, rotations = test_loader_2(ip_test_path)

            qram = get_qram_rotations(n, rotations)
            print("Path : ", ip_test_path)
            print(rotations)
            print("Statevector is : ")

            sv = execute(qram, sv_backend).result().get_statevector()
            sv = sv.to_dict()
            print(sv)

            print()

            # store to the outputs
            op_test_path = cls.op_task_path + str(test) + ".qpy"
            output_saver(op_test_path, qram)

In [ ]:
answer_gen_3.generate_answer()

#### Test `qpy`

In [196]:
circ = get_qram(4, 4, [1,2,3,4])
circ.draw()

┌───┐                                
index_0: ┤ H ├───o───────■───────o───────■────
         ├───┤   │       │       │       │    
index_1: ┤ H ├───o───────o───────■───────■────
         └───┘┌──┴───┐┌──┴───┐┌──┴───┐┌──┴───┐
value_0: ─────┤0     ├┤0     ├┤0     ├┤0     ├
              │      ││      ││      ││      │
value_1: ─────┤1 x_0 ├┤1 x_1 ├┤1 x_2 ├┤1 x_3 ├
              │      ││      ││      ││      │
value_2: ─────┤2     ├┤2     ├┤2     ├┤2     ├
              └──────┘└──────┘└──────┘└──────┘

In [ ]:
with open("test.qpy", 'wb') as f:
    qpy.dump(circ,f)
    
with open("test.qpy", 'rb') as f:
    circ2 = qpy.load(f)[0]
    
circ2.draw()